<a href="https://colab.research.google.com/github/deepshikharbhardwaj/Natural_Language_Processing/blob/main/CNN_checkpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#Hyperparameters

In [2]:
input_size = 784
in_channel = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 10
load_model = True

#Create Fully Connected Network

In [3]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3,3), stride = (1,1), padding = (1,1)) #same output with these hyperparameters
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2)) #half the output
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        return x

#Checking        
model = CNN(784, 10)
x = torch.randn(64, 1, 28, 28)
model(x).shape

torch.Size([64, 10])

# Checkpoint

In [4]:
def save_checkpoint(state, filename = "my_checkpoint.pth.tar"):
    print("=> Saving Checkpoint")
    torch.save(state, filename) #save state at filename

def load_checkpoint(checkpoint):
    print("=> Loading Checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

#Set Device

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data

In [ ]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

#Initialize Network

In [7]:
model = CNN().to(device)
# OR
#model = CNN(in_channels = in_channels, num_classes = num_classes).to(device)

#Loss and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Load Model Checkpoint

In [11]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Loading Checkpoint


#Train Network

In [10]:
for epoch in range(num_epochs): #In one epoch model has seen all the images once
    losses = []

    if epoch % 2 == 0:
        checkpoint = {'state_dict' : model.state_dict(), 'optimizer' : optimizer.state_dict()}
        save_checkpoint(checkpoint) 

    for batch_idx , (data, targets) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device)
        targets = targets.to(device)

        #CORRECT SHAPE 
        #print(data.shape) #torch.Size([64, 1, 28, 28]) 
        #data = data.reshape(data.shape[0], -1) # untouch 0th dimension

        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step() #update weights based on the gradients

=> Saving Checkpoint
=> Saving Checkpoint
=> Saving Checkpoint
=> Saving Checkpoint
=> Saving Checkpoint


#Check Accuracy

In [12]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking Accuracy on Training Data')
    else:
        print('Checking Accuracy on Test Data')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            values, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) #0th dimension in [64x10] i.e. 64
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 51665/ 60000 with accuracy 86.11 

Checking Accuracy on Test Data
Got 8738/ 10000 with accuracy 87.38 

